In [1]:
!nvidia-smi

Sun Feb 11 15:46:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000001:00:00.0 Off |                    0 |
| N/A   40C    P0              73W / 400W |  28507MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [3]:
from modeling_vision import MM_LLMs, MM_LLMs_Config
from transformers import AutoModelForCausalLM, CLIPProcessor, CLIPModel,AutoModel, AutoTokenizer, AutoProcessor,AutoConfig,CLIPConfig, LlamaConfig, WhisperConfig, WhisperModel, LlamaModel, LlamaTokenizer
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import torch
import numpy as np
from torch import nn
from streaming import LocalDataset
from typing import List

In [4]:
MM_LLMs.register_for_auto_class()
MM_LLMs_Config.register_for_auto_class()

In [5]:
from transformers.trainer_utils import get_last_checkpoint

latest = get_last_checkpoint('vision-tinyllama')
latest

'vision-tinyllama/checkpoint-3750'

In [6]:
model = MM_LLMs.from_pretrained(
    latest,flash_attention = True, dtype = torch.bfloat16, torch_dtype = torch.bfloat16
)

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour


In [7]:
_ = model.cuda()

In [8]:
image_processor = AutoProcessor.from_pretrained('google/siglip-base-patch16-384')
tokenizer = AutoTokenizer.from_pretrained(latest)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
from PIL import Image

def prepare_dataset(messages, images: List[str] = None):
    if images is not None:
        images = [Image.open(f).convert('RGB') for f in images]
        image_output = image_processor(images=images, return_tensors='pt')['pixel_values']
    else:
        image_output = None
    
    prompt = tokenizer.apply_chat_template(messages, tokenize = False)
    outputs = tokenizer(
                    prompt,
                    return_tensors='pt',
                    return_overflowing_tokens=False,
                    return_length=False
    )

    outputs['images'] = image_output
    outputs['image_index'] = torch.tensor([0] * len(outputs['images']))
    outputs['image_starts'] = torch.tensor([tokenizer.convert_tokens_to_ids('<image>')] * len(outputs['images']))
    return outputs

In [10]:
# !wget https://cdn.beautifulnara.net/wp-content/uploads/2017/12/10201620/Persian-cat-breed.jpg
# !wget https://www.jocooks.com/wp-content/uploads/2023/09/nasi-goreng-1-23.jpg

In [11]:
test_image = 'translated-LLaVA-Instruct-150K/filtered-llava-images/000000033471.jpg'
test_image2 = 'Persian-cat-breed.jpg'
test_image3 = 'abang-gay.png'
test_image4 = 'nasi-goreng-1-23.jpg'
images = [
    test_image,
    test_image2,
    test_image3,
    test_image4
]

In [12]:
for img in images:
    messages = [
        {'role': 'user', 'content': '<image> </image> ini gambar apa'},
    ]
    outputs = prepare_dataset(messages, images = [img])
    outputs['images'] = outputs['images'].type(model.dtype)
    for k in outputs.keys():
        if outputs[k] is not None:
            outputs[k] = outputs[k].cuda()

    with torch.no_grad():
        model_inputs = model.prepare_inputs_for_generation(**outputs)
    r = model_inputs.pop('input_ids', None)

    generate_kwargs = dict(
        model_inputs,
        max_new_tokens=300,
        top_p=0.95,
        top_k=50,
        temperature=0.1,
        do_sample=True,
        num_beams=1,
    )

    r = model.llm.generate(**generate_kwargs)
    print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>Imej itu menunjukkan bas penumpang putih yang besar di jalan bandar yang sibuk. Bas itu mempunyai iklan yang dipaparkan di sisinya, menarik perhatian kepada perkhidmatan pengangkutan yang ditawarkannya.

Di sekeliling bas, terdapat beberapa kereta dan trak yang diletakkan atau memandu di jalan. Terdapat juga beberapa orang yang kelihatan di tempat kejadian, ada yang berjalan dan yang lain berdiri atau duduk di kaki lima. Selain itu, terdapat dua beg tangan yang kelihatan di tempat kejadian, mungkin milik orang yang berjalan atau diletakkan di kaki lima.</s>


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>Imej itu menampilkan kucing putih yang terletak di atas sofa hitam.</s>


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>Imej itu menunjukkan seorang lelaki muda berdiri di hadapan tingkap, tersenyum dan kelihatan gembira. Dia memakai baju polo hitam dan mempunyai beg galas di sebelahnya.</s>
<s>Imej ini menampilkan mangkuk putih yang diisi dengan nasi dan pelbagai jenis makanan, termasuk lobak merah dan kacang hijau. Mangkuk itu diletakkan di atas meja, menunjukkan bahawa ia sedia untuk dimakan.</s>


In [13]:
messages = [
    {'role': 'user', 'content': '<image> </image> <image> </image> apa kaitan 2 gambar ni'},
]
outputs = prepare_dataset(messages, images = [test_image2, test_image4])
ok = outputs
ok['labels'] = ok['input_ids']

for k in ok.keys():
    if ok[k] is not None:
        ok[k] = ok[k].cuda()
        
for k in ['images']:
    if ok[k] is not None:
        ok[k] = ok[k].type(model.dtype)
        
with torch.no_grad():
    model_inputs = model.prepare_inputs_for_generation(**ok)
r = model_inputs.pop('input_ids', None)
label = model_inputs.pop('labels', None)
label = label.detach().cpu().numpy()
ok['input_ids'].shape, model_inputs['inputs_embeds'].shape

generate_kwargs = dict(
    model_inputs,
    max_new_tokens=300,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)

r = model.llm.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>Tiada hubungan langsung antara gambar 1 dan gambar 2. Gambar 1 ialah imej Anjing Kucing Perang putih, manakala gambar 2 ialah imej semangkuk nasi goreng dengan nasi, ayam dan bawang hijau. Kedua-dua imej itu tidak mempunyai sebarang elemen atau konteks yang sama.</s>


In [14]:
messages = [
    {'role': 'user', 'content': '<image> </image> abang ni nampak gay tak'},
]
outputs = prepare_dataset(messages, images = [test_image3])
ok = outputs
ok['labels'] = ok['input_ids']

for k in ok.keys():
    if ok[k] is not None:
        ok[k] = ok[k].cuda()
        
for k in ['images']:
    if ok[k] is not None:
        ok[k] = ok[k].type(model.dtype)
        
with torch.no_grad():
    model_inputs = model.prepare_inputs_for_generation(**ok)
r = model_inputs.pop('input_ids', None)
label = model_inputs.pop('labels', None)
label = label.detach().cpu().numpy()
ok['input_ids'].shape, model_inputs['inputs_embeds'].shape

generate_kwargs = dict(
    model_inputs,
    max_new_tokens=300,
    top_p=0.95,
    top_k=50,
    temperature=0.1,
    do_sample=True,
    num_beams=1,
)

r = model.llm.generate(**generate_kwargs)
tokenizer.decode(r[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s>Ya, abang kelihatan gay dalam gambar.</s>'

In [16]:
# Image.open('abang-gay.png')

In [17]:
model.push_to_hub('malaysian-tinyllama-1.1b-siglip-base-384-vision', organization='mesolitica', safe_serialization=True)

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/utils/hub.py:667: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


README.md:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/malaysian-tinyllama-1.1b-siglip-base-384-vision/commit/cbf6d2c42ed988c80d003e0f5eba47e495dcf44c', commit_message='Upload MM_LLMs', commit_description='', oid='cbf6d2c42ed988c80d003e0f5eba47e495dcf44c', pr_url=None, pr_revision=None, pr_num=None)

In [18]:
image_processor.push_to_hub('malaysian-tinyllama-1.1b-siglip-base-384-vision', organization='mesolitica', safe_serialization=True)

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/utils/hub.py:667: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/mesolitica/malaysian-tinyllama-1.1b-siglip-base-384-vision/commit/da0d3d9a93b795f21c010e3e99b952767e3aaa48', commit_message='Upload processor', commit_description='', oid='da0d3d9a93b795f21c010e3e99b952767e3aaa48', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
tokenizer.push_to_hub('malaysian-tinyllama-1.1b-siglip-base-384-vision', organization='mesolitica', safe_serialization=True)

CommitInfo(commit_url='https://huggingface.co/mesolitica/malaysian-tinyllama-1.1b-siglip-base-384-vision/commit/a338232c994d7a40e0aa89f5285034cdbdf65d6e', commit_message='Upload tokenizer', commit_description='', oid='a338232c994d7a40e0aa89f5285034cdbdf65d6e', pr_url=None, pr_revision=None, pr_num=None)